### 多模型融合训练(tensorflow)

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import h5py
import os
import shutil
import numpy as np
from sklearn.utils import shuffle

Using TensorFlow backend.


In [ ]:
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)

    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("copy/train", image_size, shuffle=False, 
                                              batch_size=16)
    val_generator = gen.flow_from_directory("copy/val", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)

    train = model.predict_generator(train_generator, train_generator.samples)
    val = model.predict_generator(val_generator, val_generator.samples)
    with h5py.File("gap_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

write_gap(ResNet50, (224, 224))
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(Xception, (299, 299), xception.preprocess_input)

In [3]:
X_train = []
X_val = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_val.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_val = np.concatenate(X_val, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [4]:
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)

D:\Programs\Anaconda2\envs\tf_py3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.




Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 8s 417us/step - loss: 0.0588 - acc: 0.9808 - val_loss: 0.0256 - val_acc: 0.9920
Epoch 2/8
20000/20000 [==============================] - 2s 116us/step - loss: 0.0216 - acc: 0.9927 - val_loss: 0.0243 - val_acc: 0.9920
Epoch 3/8
20000/20000 [==============================] - 2s 114us/step - loss: 0.0169 - acc: 0.9939 - val_loss: 0.0212 - val_acc: 0.9930
Epoch 4/8
20000/20000 [==============================] - 2s 113us/step - loss: 0.0149 - acc: 0.9952 - val_loss: 0.0220 - val_acc: 0.9932
Epoch 5/8
20000/20000 [==============================] - 2s 114us/step - loss: 0.0130 - acc: 0.9958 - val_loss: 0.0260 - val_acc: 0.9916
Epoch 6/8
20000/20000 [==============================] - 2s 113us/step - loss: 0.0117 - acc: 0.9963 - val_loss: 0.0209 - val_acc: 0.9936
Epoch 7/8
20000/20000 [==============================] - 2s 116us/step - loss: 0.0104 - acc: 0.9969 - val_loss: 0.0211 - val_ac

### 用训练的模型对测试集进行预测

In [ ]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)
print(test_generator.filenames[0])
for i, fname in enumerate(test_generator.filenames):
    fname=fname.replace('\\','/')
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)
df.head(10)